# Project 2

The goal of this assignment is for you to try out different ways of implementing and configuring a recommender, and to evaluate your different approaches.

For project 2, you’re asked to take some recommendation data (such as your toy movie dataset, Movielens, or another Dataset of your choosing), and implement at least two different recommendation algorithms on the data.  For example, content-based, user-user CF, and/or item-item CF.  You should evaluate different approaches, using different algorithms, normalization techniques, similarity methods, neighborhood sizes, etc.  You don’t need to be exhaustive—these are just some suggested possibilities.  You may use whatever third party libraries you want.  Please provide at least one graph, and a textual summary of your evaluation.

You may work in a small group.  Please submit a link to your GitHub repository for your Jupyter notebook or RMarkdown file.  Due end of day on Sunday June 26th.

**Requires the Jupyter-Scala language Kernel, available from: https://github.com/alexarchambault/jupyter-scala**

In [25]:
classpath.add( "org.apache.spark" %% "spark-core" % "1.6.1",
             "org.apache.spark" %% "spark-mllib" % "1.6.1",
              "org.apache.spark" %% "spark-sql" % "1.6.1")

0 new artifact(s)


# Response

## The Recommender System

As I'm farily new to Spark and the whole data manipulation world in Scala, let's keep the problem simple. This is a system that recommends movies to users based on the dataset collected by the class survey.

As part of this exercise, I will produce a manual similarity function and compare the performance against the collaborative filtering library in Spark

## The Code

### Firing up a Spark Context

In [26]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{StructType,StructField,StringType,IntegerType}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.{MatrixEntry, RowMatrix}

import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{StructType,StructField,StringType,IntegerType}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.linalg.distributed.{MatrixEntry, RowMatrix}

In [27]:


val conf = new SparkConf()
  .setAppName("week1-EstimatePi")
  .setMaster("local") 
val sc = new SparkContext(conf)


conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@15593650
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4af0eef9

### Data Loading and Transformations

The objective here is to:

* Load the `MovieRatings.csv` file
* Transform into Zero filled matrix
* Transform into Long-format data structure


In [28]:
// Read the CSV file
val csv = 
    sc
        .textFile("../MovieRatings.csv")
        .map(line => 
             line
                 .replaceAll(",$",", ")
                 .split(",")
                 .map(t => t.trim)
            )
csv.collect


csv: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[2] at map at Main.scala:29
res27_1: Array[Array[String]] = Array(
  Array(
    "Critic",
    "CaptainAmerica",
    "Deadpool",
    "Frozen",
    "JungleBook",
    "PitchPerfect2",
    "StarWarsForce"
  ),
  Array("Burton", "", "", "", "4", "", "4"),
  Array("Charley", "4", "5", "4", "3", "2", "3"),
  Array("Dan", "", "5", "", "", "", "5"),
  Array("Dieudonne", "5", "4", "", "", "", "5"),
  Array("Matt", "4", "", "2", "", "2", "5"),
  Array("Mauricio", "4", "", "3", "3", "4", ""),
  Array("Max", "4", "4", "4", "2", "2", "4"),
  Array("Nathan", "", "", "", "", "", "4"),
  Array("Param", "4", "4", "1", "", "", "5"),
  Array("Parshu", "4", "3", "5", "5", "2", "3"),
...

#### Transforming into Zero-filled Matrix

In [29]:
//val movies = sc.parallelize(csv.first)
val movies = csv.first
val critics = csv.collect.map(_(0))

// let's also make a parallelized version of those
val moviesPar = sc.parallelize(movies).zipWithIndex
val criticsPar = sc.parallelize(critics).zipWithIndex

movies: Array[String] = Array(
  "Critic",
  "CaptainAmerica",
  "Deadpool",
  "Frozen",
  "JungleBook",
  "PitchPerfect2",
  "StarWarsForce"
)
critics: Array[String] = Array(
  "Critic",
  "Burton",
  "Charley",
  "Dan",
  "Dieudonne",
  "Matt",
  "Mauricio",
  "Max",
  "Nathan",
  "Param",
  "Parshu",
  "Prashanth",
  "Shipra",
  "Sreejaya",
  "Steve",
  "Vuthy",
  "Xingjia"
)
moviesPar: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[4] at zipWithIndex at Main.scala:38
criticsPar: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[6] at zipWithIndex at Main.scala:41

In [30]:
val zeroFilledMatrix = 
    csv
        .collect
        .filterNot(r => r(0) == movies(0)) // filter out first row
        .map(r => r.filterNot(value => critics contains value)) // map function that returns the record minus the critic name
        .map(r => r.map(value => if (value == "") 0.00 else value.toDouble))

// and now parallelize it
val zeroFilledMatrixPar = sc.parallelize(zeroFilledMatrix)

// now, let's convert it into a linalg matrix so we can perform linear algebra operations on it
val criticMoviesMatrix = new RowMatrix(zeroFilledMatrixPar.map(line => Vectors.dense(line)))


// also, let's have a transposed version of it:
val dataTransposed =  sc.parallelize(zeroFilledMatrix.toSeq.transpose)

val moviesCriticMatrix = new RowMatrix(dataTransposed.map(line => Vectors.dense(line.toArray)))


zeroFilledMatrix: Array[Array[Double]] = Array(
  Array(0.0, 0.0, 0.0, 4.0, 0.0, 4.0),
  Array(4.0, 5.0, 4.0, 3.0, 2.0, 3.0),
  Array(0.0, 5.0, 0.0, 0.0, 0.0, 5.0),
  Array(5.0, 4.0, 0.0, 0.0, 0.0, 5.0),
  Array(4.0, 0.0, 2.0, 0.0, 2.0, 5.0),
  Array(4.0, 0.0, 3.0, 3.0, 4.0, 0.0),
  Array(4.0, 4.0, 4.0, 2.0, 2.0, 4.0),
  Array(0.0, 0.0, 0.0, 0.0, 0.0, 4.0),
  Array(4.0, 4.0, 1.0, 0.0, 0.0, 5.0),
  Array(4.0, 3.0, 5.0, 5.0, 2.0, 3.0),
  Array(5.0, 5.0, 5.0, 5.0, 0.0, 4.0),
  Array(0.0, 0.0, 4.0, 5.0, 0.0, 3.0),
  Array(5.0, 5.0, 5.0, 4.0, 4.0, 5.0),
  Array(4.0, 0.0, 0.0, 0.0, 0.0, 4.0),
  Array(4.0, 5.0, 3.0, 3.0, 3.0, 0.0),
  Array(0.0, 0.0, 5.0, 5.0, 0.0, 0.0)
)
zeroFilledMatrixPar: org.apache.spark.rdd.RDD[Array[Double]] = ParallelCollectionRDD[7] at parallelize at Main.scala:47
criticMoviesMatrix: org.apache.spark.mllib.linalg.distributed.RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@17cc6a32
dataTransposed: org.apache.spark.rdd.RDD[Seq[Double]] = ParallelCollecti

#### Transforming into a Long-format data structure

For practical purposes, we'll do an index-based long format, meaning that the string names will be substituted for an index

In [31]:
val longFormat = 
    csv
        .collect
        .filterNot(r => r(0) == movies(0)) // filter out first row
        .flatMap(r=> (1 to movies.length-1).map(i=> (r(0),movies(i-1),r(i)))) // pivot each column sothat we have: (user,movie,rating)
        .filter(r=> r._3 !="") // filter out those unrated movies
        .map(r=> (critics.indexOf(r._1),movies.indexOf(r._2),r._3.toDouble)) // convert the remaining rating to a double
                           
val ratingsLong = sc.parallelize(longFormat)

longFormat: Array[(Int, Int, Double)] = Array(
  (1, 3, 4.0),
  (1, 5, 4.0),
  (2, 0, 4.0),
  (2, 1, 5.0),
  (2, 2, 4.0),
  (2, 3, 3.0),
  (2, 4, 2.0),
  (2, 5, 3.0),
  (3, 1, 5.0),
  (3, 5, 5.0),
  (4, 0, 5.0),
  (4, 1, 4.0),
  (4, 5, 5.0),
  (5, 0, 4.0),
  (5, 2, 2.0),
  (5, 4, 2.0),
  (5, 5, 5.0),
  (6, 0, 4.0),
  (6, 2, 3.0),
...
ratingsLong: org.apache.spark.rdd.RDD[(Int, Int, Double)] = ParallelCollectionRDD[11] at parallelize at Main.scala:42

## Model Building - Cosine Distance Based

Let's now build the following models using their respective cosine distances:

* Movie-Movie collaborative filtering
* User-User collaborative filtering


### Movie-Movie  Collaborative Filtering

In [32]:
val moviesMoviesCosineDistance = criticMoviesMatrix.columnSimilarities()

val moviesMoviesSimilarities = moviesMoviesCosineDistance
  .entries
  .map {
    case MatrixEntry(i, j, u) => (i, j, u) }
  .collect
  .map(r => Seq(movies(r._1.toInt), movies(r._2.toInt), r._3.toDouble))
  .sortBy(-_(2).asInstanceOf[Double])

moviesMoviesCosineDistance: org.apache.spark.mllib.linalg.distributed.CoordinateMatrix = org.apache.spark.mllib.linalg.distributed.CoordinateMatrix@11921c07
moviesMoviesSimilarities: Array[Seq[Any]] = Array(
  List(Deadpool, Frozen, 0.9164305818234458),
  List(Critic, CaptainAmerica, 0.8011927448021527),
  List(Critic, PitchPerfect2, 0.7649201061631455),
  List(Critic, JungleBook, 0.7437115739277018),
  List(Critic, Deadpool, 0.7406840835263832),
  List(CaptainAmerica, PitchPerfect2, 0.7299810272285707),
  List(Deadpool, JungleBook, 0.7191555984642707),
  List(CaptainAmerica, Deadpool, 0.6802170238514358),
  List(Critic, Frozen, 0.5992177600023244),
  List(Frozen, JungleBook, 0.5913486717104743),
  List(CaptainAmerica, JungleBook, 0.579267135696289),
  List(Deadpool, PitchPerfect2, 0.5773720984150789),
  List(CaptainAmerica, Frozen, 0.5689794478548152),
  List(Frozen, PitchPerfect2, 0.5296627487933019),
  List(JungleBook, PitchPerfect2, 0.43479904753372806)
)

### User-User Collaborative Filtering Model

In [33]:
val userUserCosineDistance = moviesCriticMatrix.columnSimilarities()

val userUserSimilarities = userUserCosineDistance
  .entries
  .map {
    case MatrixEntry(i, j, u) => (i, j, u) }
  .collect
  .map(r => Seq(critics(r._1.toInt), critics(r._2.toInt), r._3.toDouble))
  .sortBy(-_(2).asInstanceOf[Double])

userUserCosineDistance: org.apache.spark.mllib.linalg.distributed.CoordinateMatrix = org.apache.spark.mllib.linalg.distributed.CoordinateMatrix@62e53689
userUserSimilarities: Array[Seq[Any]] = Array(
  List(Dan, Nathan, 0.9859249803487347),
  List(Mauricio, Shipra, 0.9847319278346619),
  List(Burton, Mauricio, 0.9811873171500672),
  List(Burton, Shipra, 0.9792633226865932),
  List(Burton, Parshu, 0.9610484599102903),
  List(Param, Parshu, 0.9600666937386864),
  List(Param, Shipra, 0.955672134494952),
  List(Burton, Param, 0.9474847084398104),
  List(Mauricio, Parshu, 0.9410294354946785),
  List(Mauricio, Param, 0.9296599791147713),
  List(Parshu, Shipra, 0.9293555142631518),
  List(Burton, Steve, 0.927771250724491),
  List(Dieudonne, Sreejaya, 0.9091372900969895),
  List(Prashanth, Vuthy, 0.8999999999999999),
  List(Shipra, Steve, 0.8866206949335731),
  List(Dan, Sreejaya, 0.870388279778489),
  List(Mauricio, Nathan, 0.866578244826242),
  List(Param, Steve, 0.8661218807416571),
  List(

### Querying the Models

* User: Who should Mauricio go out to the movies with?

In [34]:
val user= "Mauricio"
userUserSimilarities
    .filter(r=> r(0) == user || r(1)==user).map(r=>(if (r(1)==user) r(0) else r(1), r(2).asInstanceOf[Double]))
    .filter(_._2>0.75)
    .sortBy(-_._2)

user: String = "Mauricio"
res33_1: Array[(Any, Double)] = Array(
  (Shipra, 0.9847319278346619),
  (Burton, 0.9811873171500672),
  (Parshu, 0.9410294354946785),
  (Param, 0.9296599791147713),
  (Nathan, 0.866578244826242),
  (Steve, 0.8574929257125442),
  (Dan, 0.8123623944599234),
  (Dieudonne, 0.8081220356417687)
)

* If I liked Frozen, what other movies should I watch?

In [35]:
val movie= "Frozen"
moviesMoviesSimilarities
    .filter(r=> r(0) == movie || r(1)==movie).map(r=>(if (r(1)==movie) r(0) else r(1), r(2).asInstanceOf[Double]))
    .filter(_._2>0.75)
    .sortBy(-_._2)

movie: String = "Frozen"
res34_1: Array[(Any, Double)] = Array((Deadpool, 0.9164305818234458))

## Using Collaborative Filtering ALS Model

Based on: http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html

In [36]:
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating

val ratingsALS = ratingsLong.map(r=>Rating(r._1, r._2, r._3))

// Build the recommendation model using ALS
val rank = 10
val numIterations = 10
val model = ALS.train(ratingsALS, rank, numIterations, 0.01)

// Evaluate the model on rating data
val criticsMovies = ratingsALS.map { case Rating(critic, movie, rate) =>
  (critic, movie)
}
val predictions =
  model.predict(criticsMovies).map { case Rating(critic, movie, rate) =>
    ((critic, movie), rate)
  }
val ratesAndPredictions = ratingsALS.map { case Rating(critic, movie, rate) =>
  ((critic, movie), rate)
}.join(predictions)
val MSE = ratesAndPredictions.map { case ((critic, movie), (r1, r2)) =>
  val err = (r1 - r2)
  err * err
}.mean()

println("Mean Squared Error = " + MSE)

Mean Squared Error = 1.7300149322469586E-4


import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating
ratingsALS: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[22] at map at Main.scala:42
rank: Int = 10
numIterations: Int = 10
model: org.apache.spark.mllib.recommendation.MatrixFactorizationModel = org.apache.spark.mllib.recommendation.MatrixFactorizationModel@60b785dd
criticsMovies: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[230] at map at Main.scala:54
predictions: org.apache.spark.rdd.RDD[((Int, Int), Double)] = MapPartitionsRDD[240] at map at Main.scala:59
ratesAndPredictions: org.apache.spark.rdd.RDD[((Int, Int), (Double, Double))] = MapPartitionsRDD[244] at join at Main.scala:66
MSE: Double = 1.7300149322469586E-4

In [37]:
model.recommendProducts(critics.indexOf(user),5).map(r => (critics(r.user), movies(r.product), r.rating)) 
model.recommendUsersForProducts(3).collect.flatMap(m=>m._2.map(r=>(movies(m._1),critics(r.user),r.rating)))
model.recommendProductsForUsers(3).collect.flatMap(m=>m._2.map(r=>(movies(r.product),critics(m._1),r.rating)))


res36_0: Array[(String, String, Double)] = Array(
  ("Mauricio", "CaptainAmerica", 4.55839706531505),
  ("Mauricio", "Critic", 4.000934381708938),
  ("Mauricio", "JungleBook", 3.9826942547896538),
  ("Mauricio", "PitchPerfect2", 3.896175989914277),
  ("Mauricio", "Deadpool", 3.0055283089099647)
)
res36_1: Array[(String, String, Double)] = Array(
  ("JungleBook", "Mauricio", 3.9826942547896538),
  ("JungleBook", "Sreejaya", 3.981912103579872),
  ("JungleBook", "Vuthy", 2.9946114143957767),
  ("Critic", "Sreejaya", 5.015867105191404),
  ("Critic", "Dieudonne", 4.985885047261088),
  ("Critic", "Prashanth", 4.9770757945934605),
  ("CaptainAmerica", "Sreejaya", 5.006493164406723),
  ("CaptainAmerica", "Prashanth", 5.001166196717815),
  ("CaptainAmerica", "Dan", 4.999004400387928),
  ("Frozen", "Parshu", 5.002470638416293),
  ("Frozen", "Xingjia", 4.999274699833141),
  ("Frozen", "Prashanth", 4.998333628212166),
  ("PitchPerfect2", "Dieudonne", 5.000081591642868),
  ("PitchPerfect2", "Dan", 

In [38]:
//sc.stop

In [50]:
val moviesCriticMatrix = new RowMatrix(sc.parallelize(Array(Array(1.00,4.0),Array(1.0,4.0))).map(line => Vectors.dense(line)))

moviesCriticMatrix: org.apache.spark.mllib.linalg.distributed.RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@41baebb1

In [51]:
moviesCriticMatrix.columnSimilarities().entries.collect

res50: Array[org.apache.spark.mllib.linalg.distributed.MatrixEntry] = Array(MatrixEntry(0L, 1L, 0.9999999999999998))